## corv: Validations Off Of Falcon et. al, 2010

[GitHub Page](https://github.com/vedantchandra/corv)

---

Table Of Contents:

1. [Validation Without Templates](#validation-no-template)
2. [Validation With Templates](#db-no-template)

---

In order to establish that `corv` actually works, we can compare the radial velocities it calculates to DA radial velocities calculated from high-resolution spectra by [Falcon et.al, 2010](https://ui.adsabs.harvard.edu/abs/2010ApJ...712..585F/abstract). We use the sample of DA white dwarfs with spectra from [Chandra et.al, 2020](https://iopscience.iop.org/article/10.3847/1538-4357/aba8a2) and cross-match with the Falcon catalog based on position. Then we run `corv` on each spectrum, and compare the two.

<a id="validation-no-template"></a>

**01. Validating RVs Without Templates (Need to check/proofread)**

---

First we validate `corv` in the simplest case: without template fitting.

In [1]:
### General
import numpy as np
import matplotlib.pyplot as plt

from astropy.table import Table, Column, MaskedColumn, join
from astroquery.sdss import SDSS

from tqdm import tqdm

import corv
#corv.sdss.make_catalogs()

/Users/vedantchandra/0_research/01_sdss5/006_build_corv/data/comm_cat/
star and exposure catalogs not found! check paths and run make_catalogs() if you want to use sdss functionality. otherwise ignore.


In [2]:
# Read in the catalogs for comparison
catalog = Table.read('data/sed_radii.fits')
falcon = Table.read('data/falcon2010.fit')

# Initialize the corv rv arrays
catalog['corv_rv'] = -9999 * np.ones(len(catalog))
catalog['corv_erv'] = -9999 * np.ones(len(catalog))

In [3]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import  join_skycoord
from astropy import table

# Create a column with SkyCoord positions for easy matching
catalog['wd_pos'] = SkyCoord(catalog['ra'], catalog['dec'], unit='deg')
falcon['wd_pos'] = SkyCoord(falcon['_RA'], falcon['_DE'], unit='deg')

# Match Falcon's and Vedant's catalogs by taking targets within 5 arcsec
join_func = table.join_skycoord(5 * u.arcsecond)
falcon_xmatch = table.join(catalog, falcon, join_funcs={'wd_pos': join_skycoord(5 * u.arcsec)})

In [4]:
figs = []

for j in tqdm( range(len(falcon_xmatch))):
    # Extract SDSS plate, mjd, and fiberID from each target in both catalogs
    p,m,f = np.array(falcon_xmatch['col_p_m_f'][j].split('-')).astype(float)
    
    # Query SDSS for spectra 
    xid = SDSS.query_specobj(plate = p, mjd = m, fiberID = f)
    sp = SDSS.get_spectra(matches=xid)
    
    # Get pertinent spectrum data    
    wl = np.array(10**sp[0][1].data['loglam'])
    fl = np.array(sp[0][1].data['flux'])
    ivar = np.array(sp[0][1].data['ivar'])
            
    # Make & fit a corv model without templates
    corvmodel = corv.models.make_balmer_model(nvoigt = 2, names = ['a','b','g','d'])
    param_res, rv_res, rv_init = corv.fit.fit_corv(wl, fl, ivar, corvmodel)        
        
    # Add that to the list
    falcon_xmatch['corv_rv'][j] = (rv_res.params['RV'].value)
    falcon_xmatch['corv_erv'][j] = (rv_res.params['RV'].stderr)

  0%|                                                                                            | 0/97 [00:00<?, ?it/s]/home/arseneau/anaconda3/lib/python3.9/site-packages/astroquery/sdss/core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
Traceback (most recent call last):
  File "/home/arseneau/anaconda3/lib/python3.9/site-packages/emcee/ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/home/arseneau/anaconda3/lib/python3.9/site-packages/lmfit/minimizer.py", line 1120, in _lnprob
    out = userfcn(params, *userargs, **userkwargs)
  File "/home/arseneau/anaconda3/lib/python3.9/site-packages/corv-0.1-py3.9.egg/corv/fit.py", line 173, in <lambda>
    residual = lambda params: normalized_residual(wl, fl, ivar,
  File "/home/arseneau/anaconda3/lib/python3.9/site-p

emcee: Exception while calling your likelihood function:
  params: [8.71742883e+01 1.00001992e-02]
  args: (<function fit_rv.<locals>.<lambda> at 0x7f87dcb45dc0>, Parameters([('c', <Parameter 'c', value=0.9246038166171139 (fixed), bounds=[-inf:inf]>), ('a0_amplitude', <Parameter 'a0_amplitude', value=12.579217955550149 (fixed), bounds=[0:inf]>), ('a0_center', <Parameter 'a0_center', value=6566.519148865587, bounds=[-inf:inf], expr='6564.61/ sqrt((1 - RV/2.99792458e5)/(1 + RV/2.99792458e5))'>), ('a0_sigma', <Parameter 'a0_sigma', value=15.502554578699243 (fixed), bounds=[0:inf]>), ('a1_amplitude', <Parameter 'a1_amplitude', value=2.104187100976411 (fixed), bounds=[0:inf]>), ('a1_center', <Parameter 'a1_center', value=6566.519148865587, bounds=[-inf:inf], expr='a0_center'>), ('a1_sigma', <Parameter 'a1_sigma', value=4.30523904600564 (fixed), bounds=[0:inf]>), ('b0_amplitude', <Parameter 'b0_amplitude', value=3.0640429801614024 (fixed), bounds=[0:inf]>), ('b0_center', <Parameter 'b0_cente

KeyboardInterrupt: 

In [ ]:
rms = (falcon_xmatch['Adp-V'][np.abs(falcon_xmatch['corv_rv']) < 2500] - falcon_xmatch['corv_rv'][np.abs(falcon_xmatch['corv_rv']) < 2500])**2

In [ ]:
def linear(x):
    return x

plt.figure(figsize=(10,10))
plt.style.use('stefan.mplstyle')

plt.errorbar(falcon_xmatch['Adp-V'][np.abs(falcon_xmatch['corv_rv']) < 2500], falcon_xmatch['corv_rv'][np.abs(falcon_xmatch['corv_rv']) < 2500], 
             xerr = falcon_xmatch['e_Adp-V'][np.abs(falcon_xmatch['corv_rv']) < 2500], yerr = falcon_xmatch['corv_erv'][np.abs(falcon_xmatch['corv_rv']) < 2500],
             fmt='o', label = 'Data', color='black', ecolor = 'teal')
plt.plot(falcon_xmatch['vr'][np.abs(falcon_xmatch['corv_rv']) < 2500], linear(falcon_xmatch['vr'][np.abs(falcon_xmatch['corv_rv']) < 2500]), color = 'black')


ymin, ymax = plt.ylim()
plt.xlabel(r'RV (Falcon et. al, 2010) $[km/s]$')
plt.ylabel(r'RV (corv) $[km/s]$')

<a id="validation-template"></a>

**01. Validating RVs With Templates (Need to check/proofread)**

---

In [ ]:
figs = []

for j in tqdm( range(len(falcon_xmatch))):
    p,m,f = np.array(falcon_xmatch['col_p_m_f'][j].split('-')).astype(float)
    
    try:
        xid = SDSS.query_specobj(plate = p, mjd = m, fiberID = f)
    except ValueError:
        print('unknown error')
        
    try:
        sp = SDSS.get_spectra(matches=xid)
    except:
        print('http error')
        continue
    for i in range(len(sp[0:1])):
        wl = np.array(10**sp[i][1].data['loglam'])
        fl = np.array(sp[i][1].data['flux'])
        ivar = np.array(sp[i][1].data['ivar'])
                
        corvmodel = corv.models.make_koester_model(names = ['a','b','g','d'])
        param_res, rv_res, rv_init = corv.fit.fit_corv(wl, fl, ivar, corvmodel)        
        
        
    falcon_xmatch['corv_rv'][j] = (rv_res.params['RV'].value)
    falcon_xmatch['corv_erv'][j] = (rv_res.params['RV'].stderr)

In [ ]:
plt.figure(figsize=(10,10))
plt.style.use('stefan.mplstyle')

plt.errorbar(falcon_xmatch['Adp-V'][np.abs(falcon_xmatch['corv_rv']) < 2500], falcon_xmatch['corv_rv'][np.abs(falcon_xmatch['corv_rv']) < 2500], 
             xerr = falcon_xmatch['e_Adp-V'][np.abs(falcon_xmatch['corv_rv']) < 2500], yerr = falcon_xmatch['corv_erv'][np.abs(falcon_xmatch['corv_rv']) < 2500],
             fmt='o', label = 'Data', color='black', ecolor = 'teal')
plt.plot(falcon_xmatch['vr'][np.abs(falcon_xmatch['corv_rv']) < 2500], linear(falcon_xmatch['vr'][np.abs(falcon_xmatch['corv_rv']) < 2500]), color = 'black')


ymin, ymax = plt.ylim()
plt.xlabel(r'RV (Falcon et. al, 2010) $[km/s]$')
plt.ylabel(r'RV (corv) $[km/s]$')